In [42]:
import os
import cv2
import copy
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [64]:
def seg_switcher(seg_img):
    color_mapping = {
        (255, 255, 170): (10,10,10),  # hair
        (255, 255, 255): ( 0,0,0),  # background
        (0, 255, 170): (0,0,0) ,      # [0, 255, 170] earring to [0,0,0] background
        (255, 255,   0): (13,13,13),      #  [255, 255, 0] necklace to [0, 170, 255] neck
        (255, 255,  85): (0,0,0),    # [255, 255,  85] clothes to [0,0,0] background
        (0, 170, 255): (13,13,13),      # neck
        ( 0, 255,  85 ): (12,12,12),      # right ear
        (170, 255,   0): (11,11,11),             # left ear
        (0, 255, 0): (5,5,5),          # right eye
        (255, 0, 85): (3,3,3),        # right eyebrow
        (255, 170, 0): (2,2,2),      # left eyebrow
        (0, 0, 255): (6,6,6),          # nose
        (255, 85, 0): (1,1,1),        # face
        (255, 0, 170): (4,4,4),      # left eye
        (0, 85, 255): (9,9,9),        # lower lip
        (170, 0, 255): (7,7,7),      # upper lip
        (85,  0, 255):(8,8,8),
        (255, 0, 255):(0,0,0)    # hat to background
    }

    # Replace old colors with new colors
    for old_color, new_color in color_mapping.items():
        seg_img[np.all(seg_img == old_color, axis=-1)] = new_color
    return seg_img

def seg_corrector(cv2_image_masked):

    height, width,_ = cv2_image_masked.shape
    right_half_face = cv2_image_masked[:, :width//2, :]
    left_half_face = cv2_image_masked[:, width//2:, :]

    color_mapping_ltr = {
        (255, 0, 170): (0, 255, 0),       #left_eye color to right_eye
        (255, 170, 0): (255, 0, 85)       #left_eyebrow color to right_eyebrow
    }
    color_mapping_rtl = {
        (0, 255, 0): (255, 0, 170),       #right_eye color to left_eye
        (255, 0, 85): (255, 170, 0)       #right_eyebrow color to left_eyebrow
    }
    for old_color, new_color in color_mapping_ltr.items():
        right_half_face[np.all(right_half_face == old_color, axis=-1)] = new_color
    for old_color, new_color in color_mapping_rtl.items():
        left_half_face[np.all(left_half_face == old_color, axis=-1)] = new_color

    original_img = np.concatenate((right_half_face, left_half_face), axis=1)
    return original_img

def resize(new_width, new_height,input_img):
    pil_image = Image.fromarray(input_img.astype(np.uint8))
    resized_image = pil_image.resize((new_width, new_height), Image.NEAREST)
    return resized_image

def check_nums_class(seg_img):
    # Reshape the image to a 2D array where each row represents a pixel
    pixels = seg_img.reshape(-1, seg_img.shape[2])

    # Find unique colors (combinations of RGB values)
    unique_colors = np.unique(pixels, axis=0)

    # Convert the unique colors to a list of tuples
    unique_colors_list = [tuple(color) for color in unique_colors]

    return unique_colors_list

def img_group_class_check(checking_folder,file_list):
    channel_value = {}
    channel_value = set()
    img_checker = {}
    img_checker = set()
    for i in file_list:
        img = cv2.imread(get_path(checking_folder,i))
        img_classes = check_nums_class(img)
        channel_value.add(len(img_classes))
        if len(img_classes)>20:
            img_checker.add(i)
    return channel_value, img_checker

def list_files_in_folder(folder_path):
    file_list = []
    # Iterate through all files in the folder
    for file_name in os.listdir(folder_path):
        # Check if the path is a file (not a directory)
        if os.path.isfile(os.path.join(folder_path, file_name)):
            file_list.append(file_name)
    return file_list

def get_path(folderPath,fileName):
    path = os.path.join(folderPath,fileName)
    return path

def iterate(file_list,input_folder,output_folder):
    for i in file_list:
        print(i)
        img = cv2.imread(get_path(input_folder,i))
        corrector_output = seg_corrector(img)
        switcher_output = seg_switcher(corrector_output)
        final_output = resize(321,321,switcher_output)
        final_output.save(get_path(output_folder,i))
        # cv2.imwrite(get_path(output_folder,i),corrector_output)

In [41]:
%reset -f

In [ ]:
seg_img = cv2.imread(r"D:\Project\DPL302m\faceSegmantation\maskModifier\seg_final\makeup\023.png")
# corrector_output = seg_corrector(seg_img)
# switcher_output = seg_switcher(corrector_output)
# final_output = resize(321,321,switcher_output)
channel = check_nums_class(seg_img)
len(channel)
# seg_img = seg_switcher(seg_img)
# cv2.imshow('Modified Segmentation Image', seg_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# cv2.imwrite(r'D:\Project\DPL302m\faceSegmantation\maskModifier\fixOutput.png', seg_img)


In [66]:
folder_seg = r"D:\Project\DPL302m\Data\crawledData2\generatorOutput\seg\beforeFixed\non_makeup\not_good"
folder_seg_painted = r"D:\Project\DPL302m\faceSegmantation\maskModifier\seg_paint\makeup"
folder_seg_resized = r"D:\Project\DPL302m\faceSegmantation\maskModifier\seg_resized\makeup"
folder_seg_bw = r"D:\Project\DPL302m\faceSegmantation\maskModifier\seg_final\makeup"
output_folder = r"D:\Project\DPL302m\faceSegmantation\maskModifier\seg_final\non_makeup\notGood"

file_list = list_files_in_folder(folder_seg)
iterate(file_list,folder_seg,output_folder)

# file_list = list_files_in_folder(output_folder)
# diff_channel_value, img_checker = img_group_class_check(output_folder,file_list)


003.png
008.png
012.png
014.png
020.png
022.png
025.png
036.png
037.png
039.png
057.png
081.png
082.png
090.png
091.png
092.png
104.png
106.png
107.png
112.png
115.png
118.png
119.png
121.png
122.png
123.png
134.png
162.png


In [56]:
diff_channel_value
# img_checker

{10, 11, 12, 13, 14}

In [ ]:
def resize_nearest_neighbor(input_image_path, new_width, new_height):
    # Open the input image
    input_image = Image.open(input_image_path)

    # Resize the image using nearest neighbor interpolation
    output_image = input_image.resize((new_width, new_height), Image.NEAREST)
    return output_image
    # Save the resized image
    output_image.save(output_image_path)

input_image_path = r"D:\Project\DPL302m\faceSegmantation\maskModifier\seg_final\512x512makeup\185.png"
output_image_path = r"D:\Project\DPL302m\faceSegmantation\output.png"
new_width = 321
new_height = 321
new_img = resize_nearest_neighbor(input_image_path, new_width, new_height)
copy_seg_img = copy.deepcopy(new_img)
# Define the color to be replaced and the color to replace it with
color_to_replace = np.array([9,9,9])
replacement_color = np.array([255,255,255])

# # Find pixels with the color to be replaced
mask = np.all(copy_seg_img == color_to_replace, axis=-1)

# # Replace pixels with the replacement color
copy_seg_img[mask] = replacement_color
plt.imshow(copy_seg_img)